**RLeXplore allows you to combine multiple intrinsic rewards to explore the potential  assembly advantages.**

**Load the libraries**

In [1]:
import torch as th

from rllte.env import make_atari_env
from rllte.xplore.reward import Fabric, RE3, ICM
from rllte.agent import PPO

**Use the `Fabric` class to create a mixed intrinsic reward**

In [2]:
class TwoMixed(Fabric):
    def __init__(self, m1, m2):
        super().__init__(m1, m2)
    
    def compute(self, samples, sync):
        rwd1, rwd2 = super().compute(samples, sync)

        return rwd1 + rwd2

class ThreeMixed(Fabric):
    def __init__(self, m1, m2, m3):
        super().__init__(m1, m2, m3)
    
    def compute(self, samples, sync):
        rwd1, rwd2, rw3 = super().compute(samples, sync)

        return (rwd1 + rwd2) * rw3

In [3]:
# set the parameters
device = 'cuda' if th.cuda.is_available() else 'cpu'
# create the vectorized environments
envs = make_atari_env(env_id='BreakoutNoFrameskip-v4', device=device, num_envs=8)
# create two intrinsic reward functions
irs1 = ICM(envs, device)
irs2 = RE3(envs, device)
# create the mixed intrinsic reward function
irs = TwoMixed(irs1, irs2)

In [1]:
# create the PPO agent
agent = PPO(envs, device=device)
# set the intrinsic reward module
agent.set(reward=irs)
# train the agent
agent.train(10000)